# 卷积神经网络对图片进行分类。kaggle

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib as mpl
import os,time
import numpy as np
import pandas as pd
import sklearn
import sys
import tensorflow as tf
from tensorflow import keras

print(sys.version_info)
for module in mpl,np,pd,sklearn,tf,keras:
    print(module.__name__,module.__version__)


sys.version_info(major=3, minor=6, micro=9, releaselevel='final', serial=0)
matplotlib 3.2.0
numpy 1.18.1
pandas 1.0.1
sklearn 0.22.2.post1
tensorflow 2.1.0
tensorflow_core.python.keras.api._v2.keras 2.2.4-tf


In [2]:
tf.config.list_physical_devices('GPU')

[]

In [11]:
class_names = [
        'airplane',
        'automobile',
        'bird' ,
        'cat' ,
        'deer' ,
        'dog' ,
        'frog' ,
        'horse' ,
        'ship' ,
        'truck',
        ]
train_labels_file = 'cifar10/trainLabels.csv'
test_csv_file = 'cifar10/sampleSubmission.csv'
train_folder = 'cifar10/train/'
test_folder = 'cifar10/rest'

def parse_csv_file(filepath, folder):
    """Parses csv files into (filename(full_path), label) format"""
    results = []
    with open(filepath, 'r') as f:
        lines = f.readlines()
    for line in lines:
        image_id, label_str = line.strip('\n').split(',')
        image_full_path = os.path.join(folder, image_id + '.png')
        results.append((image_full_path,label_str))
    return results
train_labels_info = parse_csv_file(train_labels_file, train_folder)
test_csv_info = parse_csv_file(test_csv_file,test_folder)

import pprint
pprint.pprint(train_labels_info[:5])
pprint.pprint(test_csv_info[:5])
print(len(train_labels_info), len(test_csv_info))

[('cifar10/train/id.png', 'label'),
 ('cifar10/train/1.png', 'frog'),
 ('cifar10/train/2.png', 'truck'),
 ('cifar10/train/3.png', 'truck'),
 ('cifar10/train/4.png', 'deer')]
[('cifar10/rest/id.png', 'label'),
 ('cifar10/rest/1.png', 'cat'),
 ('cifar10/rest/2.png', 'cat'),
 ('cifar10/rest/3.png', 'cat'),
 ('cifar10/rest/4.png', 'cat')]
50001 300001


In [5]:
# train_df = pd.Dataframe(train_labels_info)
train_df = pd.DataFrame(train_labels_info[1:45001])
valid_df = pd.DataFrame(train_labels_info[45001:])
test_df = pd.DataFrame(test_csv_info[1:])

train_df.columns = ['filepath', 'class']
valid_df.columns = ['filepath', 'class']
test_df.columns = ['filepath', 'class']

print(train_df.head())
print(train_df.shape)
print(test_df.head())
print(test_df.shape)
print(valid_df.head())
print(valid_df.shape)


              filepath       class
0  cifar10/train/1.png        frog
1  cifar10/train/2.png       truck
2  cifar10/train/3.png       truck
3  cifar10/train/4.png        deer
4  cifar10/train/5.png  automobile
(45000, 2)
             filepath class
0  cifar10/rest/1.png   cat
1  cifar10/rest/2.png   cat
2  cifar10/rest/3.png   cat
3  cifar10/rest/4.png   cat
4  cifar10/rest/5.png   cat
(300000, 2)
                  filepath       class
0  cifar10/train/45001.png       horse
1  cifar10/train/45002.png  automobile
2  cifar10/train/45003.png        deer
3  cifar10/train/45004.png  automobile
4  cifar10/train/45005.png    airplane
(5000, 2)


In [6]:
height = 32# 定义将图片缩放到同样大小图片大小
width = 32 
channels = 3# 彩色图片，3通道
batch_size = 32 # 生产的图片多少张为一组
num_classes = 10# 类别的个数
# ImageDataGenerator读取图片并作数据增强
# 1、初始化一个generator
train_datagen = keras.preprocessing.image.ImageDataGenerator(
    rescale = 1./255,
    rotation_range = 40,# 将图片随机旋转角度，40指0-40之间随机选一个数
    width_shift_range= 0.2,# 对数据进行位移小于1代表最大比例，大于1代表最大像素值
    height_shift_range =0.2,
    shear_range=0.2,# 剪切强度
    zoom_range=0.2,# 缩放强度
    horizontal_flip= True,# 随机水平翻转
    fill_mode='nearest',# 如果填充像素，采用的方法
)
# 2、读取图片
train_generator = train_datagen.flow_from_dataframe(train_df,
#                                                    directory = './',
                                                   x_col='filepath',
                                                   y_col = 'class',
                                                   classes= class_names,
                                                   target_size = (height, width),
                                                   batch_size = batch_size,
                                                   seed = 7,
                                                   shuffle = True,
                                                   class_mode = 'sparse'
                                                   )
valid_datagen = keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
#  验证集一般不对图像处理，只要缩放值就可以
valid_generator = valid_datagen.flow_from_dataframe(valid_df,
#                                                    directory = './',
                                                   x_col = 'filepath',
                                                   y_col = 'class',
                                                   classes = class_names,
                                                   target_size=(height,width),
                                                   batch_size=batch_size,
                                                   seed = 7,
                                                   shuffle= False,
                                                   class_mode = 'sparse')

train_num = train_generator.samples#查看数据数量
valid_num = valid_generator.samples
print(train_num,valid_num)

Found 45000 validated image filenames belonging to 10 classes.
Found 5000 validated image filenames belonging to 10 classes.
45000 5000


In [7]:
for i in range(2):
    x,y = train_generator.next()
    print(x.shape,y.shape)
    print(y)

(32, 32, 32, 3) (32,)
[2. 1. 4. 4. 4. 4. 6. 5. 2. 8. 4. 6. 6. 3. 7. 1. 7. 2. 8. 8. 3. 0. 5. 3.
 9. 1. 4. 5. 6. 7. 9. 2.]
(32, 32, 32, 3) (32,)
[0. 7. 2. 7. 5. 5. 7. 0. 5. 4. 9. 7. 6. 3. 0. 4. 4. 4. 6. 3. 5. 4. 6. 6.
 4. 1. 8. 2. 4. 4. 3. 0.]


In [8]:

model = keras.models.Sequential([
    keras.layers.Conv2D(filters=128,kernel_size=3,padding='same',
                       activation='selu',input_shape=[width,height,channels]),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2D(filters=128,kernel_size=3,padding='same',
                           activation='selu'),
    keras.layers.BatchNormalization(),#批归一化
    keras.layers.MaxPool2D(pool_size=2),#MaxPool，Avgpool

    keras.layers.Conv2D(filters=256,kernel_size=3,padding='same',
                       activation='selu'),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2D(filters=256,kernel_size=3,padding='same',
                       activation='selu'),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=2),

    keras.layers.Conv2D(filters=512,kernel_size=3,padding='same',
                       activation='selu'),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2D(filters=512,kernel_size=3,padding='same',
                       activation='selu'),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=2),
    keras.layers.Flatten(),
    keras.layers.Dense(128,activation='selu'),
    keras.layers.Dense(num_classes,activation='softmax')
])
#由于class_mode是sparse，所以使用sparse_categorical_crossentropy
model.compile(loss ='sparse_categorical_crossentropy',
             optimizer='adam',metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 32, 32, 128)       3584      
_________________________________________________________________
batch_normalization (BatchNo (None, 32, 32, 128)       512       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 128)       147584    
_________________________________________________________________
batch_normalization_1 (Batch (None, 32, 32, 128)       512       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 16, 16, 128)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 16, 16, 256)       295168    
_________________________________________________________________
batch_normalization_2 (Batch (None, 16, 16, 256)       1

In [10]:
epochs = 2
# 数据由generator生成，所以用fit_generator
history = model.fit(train_generator,
                             steps_per_epoch=train_num//batch_size,
                              epochs= epochs,
                          validation_data = valid_generator,
                              validation_steps = valid_num//batch_size
                             )

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 1406 steps, validate for 156 steps
Epoch 1/2
 459/1406 [========>.....................] - ETA: 23:27 - loss: 2.3735 - accuracy: 0.2516

KeyboardInterrupt: 

In [12]:
print(history.history.keys())

NameError: name 'history' is not defined

In [ ]:
# 将整个模型保存为HDF5文件
model.save('cifar10/cafar10_model_1.h5')

In [ ]:
data={}
def plot_learning_curves(history,label, epochs, min_value, max_value):
    data[label] = history.history[label]
    data['val_'+label] = history.history['val_'+label]
    pd.DataFrame(data).plot(figsize = (8,5))
    plt.grid(True)
#     plt.axis([0,epochs],[min_value,max_value])
    
plot_learning_curves(history,'accuracy',epochs,0,1)
plot_learning_curves(history,'loss',epochs,1,2.5)

In [ ]:
test_datagen = keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
#  验证集一般不对图像处理，只要缩放值就可以
test_generator = valid_datagen.flow_from_datafram(test_df,
                                                   directory = './',
                                                   x_col = 'filepath',
                                                   y_col = 'class',
                                                   classes = class_names,
                                                   target_size=(height,width),
                                                   batch_size=batch_size,
                                                   seed = 7,
                                                   shuffle= False,
                                                   class_mode = 'sparse')
test_num = test_generator.samples
print(test_num)

In [ ]:
test_predict = model.predict_generator(test_generator,
                                                      workers = 10，#并行处理量
                                                        use_multiprecessing = True#多进程，false多线程
                                                 )

In [ ]:
print（test_predict.shape）

In [ ]:
print(test_predict[0:5])

In [ ]:
test_predict_class_indices = np.argmax(test_predict, axis = 1)
print(test_predict_class_indices[:5])

In [ ]:
test_predict_class = [class_names[index] for index in test_predict_class_indices]
print(test_predict_class[:5])

In [ ]:
def generate_submissions(filename, predict_class):
    with open(filename, 'w') as f:
        f.write('id,label\n')
        for i in range(len(predict_class)):
            f.write('%d,%s\n' %(i+1, predict_class[i]))
            
output_file = 'data/submission.csv'
generate_submissions(output_file, test_predict_class)

In [ ]:
# https://www.kaggle.com/c/cifar-10/data
# late submossion上传submission.csv文件
# 描述文件，写主要参数，
# eg.
# 'train 45000 20 epichs, basic model with only conv + bn + pooling'